In [1]:
# get environment variables
import os
import sys
sys.path.append(os.environ.get('srcdir'))

# always reload modules marked with "%aimport"
%load_ext autoreload
%autoreload 1

from pyresample import kd_tree, geometry
from features.log_progress import log_progress
import numpy as np
import xarray as xr

In [2]:
import xarray as xr
import matplotlib.pyplot as plt

hr_grd_path = os.path.join(os.environ.get("intdir"),'waom1_grd_raw.nc')
hr_grd = xr.open_dataset(hr_grd_path)

lr_grd_path = os.path.join(os.environ.get("prodir"),'waom10_grd.nc')
lr_grd = xr.open_dataset(lr_grd_path)

#hr_sbc_path =  os.path.join('/home/ubuntu/raijin/g/data2/gh9/oxr581/waom1_frc',var+'_'+month+'.nc')
#outpath = os.path.join(os.environ.get('prodir'),'waom2_'+var+'_'+month+'.nc')

# sbc

In [3]:
def resample_frc(hr_grd,lr_grd,hr_sbc,var,gt,time_coord,write=False):

    lr_def = geometry.SwathDefinition(lons=lr_grd['lon_'+gt].values,lats=lr_grd['lat_'+gt].values)
    hr_def = geometry.SwathDefinition(lons=hr_grd['lon_'+gt].values,lats=hr_grd['lat_'+gt].values)

    lr_sbc = hr_sbc.drop(var)
    dummy = np.empty((lr_sbc[time_coord].size,lr_grd['eta_'+gt].size,lr_grd['xi_'+gt].size))
    lr_sbc[var] = xr.DataArray(dummy,dims=[time_coord,'eta_'+gt,'xi_'+gt])

    valid_input_index, valid_output_index, index_array, distance_array = \
    kd_tree.get_neighbour_info(hr_def,lr_def, 2000,neighbours=4,nprocs=16)

    wf = lambda r: 1/r

    for day in log_progress(range(hr_sbc[time_coord].size),name="day"):
    
        lr_sbc[var][day] = kd_tree.get_sample_from_neighbour_info('custom', lr_def.shape, hr_sbc[var][day].values,\
                                                 valid_input_index,\
                                                 valid_output_index,index_array,distance_array,wf)

    lr_sbc[time_coord].attrs = hr_sbc[time_coord].attrs
    lr_sbc[var].attrs = hr_sbc[var].attrs
    
    if write:
        outpath = os.path.join(os.environ.get('prodir'),'waom2_'+var+'.nc')
        lr_sbc.to_netcdf(outpath,unlimited_dmn=time_coord)
    
    return lr_sbc

In [8]:
#for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
for month in ['12']:
    print('processing month ',month)
    for var,gt,time in zip(['shflux','swflux','sustr','svstr'],['rho','rho','u','v'],['shf_time','swf_time','sms_time','sms_time']):
        print('processing ',var)
        hr_sbc_path =  os.path.join('/home/ubuntu/raijin/g/data2/gh9/oxr581/waom1_frc',var+'_'+month+'.nc')
        hr_sbc = xr.open_dataset(hr_sbc_path)
        lr_sbc = resample_frc(hr_grd,lr_grd,hr_sbc,var,gt,time)
        outpath = os.path.join(os.environ.get('intdir'),'waom10_'+var+'_'+month+'.nc')
        lr_sbc.to_netcdf(outpath,unlimited_dims=time)
        del lr_sbc

processing month  12
processing  shflux


/home/ubuntu/bigStick/anaconda3/envs/tidal_melting/lib/python3.6/site-packages/pyresample/kd_tree.py:399: UserWarning: Possible more than 4 neighbours within 2000 m for some data points
  (neighbours, radius_of_influence))


VBox(children=(HTML(value=''), IntProgress(value=0, max=31)))

processing  swflux


VBox(children=(HTML(value=''), IntProgress(value=0, max=31)))

processing  sustr


VBox(children=(HTML(value=''), IntProgress(value=0, max=31)))

processing  svstr


VBox(children=(HTML(value=''), IntProgress(value=0, max=31)))

# nudge

In [4]:
hr_nudge_path = os.path.join(os.environ.get('prodir'),'waom1_nudge.nc')
hr_nudge = xr.open_dataset(hr_nudge_path)
hr_nudge

<xarray.Dataset>
Dimensions:   (eta_rho: 5300, sss_time: 12, sst_time: 12, xi_rho: 6300)
Coordinates:
  * sss_time  (sss_time) timedelta64[ns] 15 days 04:59:59.971200 ...
  * sst_time  (sst_time) timedelta64[ns] 15 days 04:59:59.971200 ...
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    SSS       (sss_time, eta_rho, xi_rho) float64 ...
    SST       (sst_time, eta_rho, xi_rho) float64 ...
    dQdSST    (sst_time, eta_rho, xi_rho) float64 ...

In [5]:
lr_nudge = {}
for var,gt,time in zip(['SSS','SST','dQdSST'],['rho','rho','rho'],['sss_time','sst_time','sst_time']):
    print('processing ',var)
    lr_nudge[var] = resample_frc(hr_grd,lr_grd,hr_nudge[var].to_dataset(),var,gt,time,write=False)
lr_nudge = xr.merge([lr_nudge['SSS'],lr_nudge['SST'],lr_nudge['dQdSST']]) 
lr_nudge.attrs = hr_nudge.attrs
lr_out_path = os.path.join(os.environ.get('prodir'),'waom10_nudge.nc')
lr_nudge.to_netcdf(lr_out_path)  

processing  SSS


/home/ubuntu/bigStick/anaconda3/envs/tidal_melting/lib/python3.6/site-packages/pyresample/kd_tree.py:399: UserWarning: Possible more than 4 neighbours within 2000 m for some data points
  (neighbours, radius_of_influence))


VBox(children=(HTML(value=''), IntProgress(value=0, max=12)))

processing  SST


VBox(children=(HTML(value=''), IntProgress(value=0, max=12)))

processing  dQdSST


VBox(children=(HTML(value=''), IntProgress(value=0, max=12)))

# tds

In [6]:
hr_tds_path = os.path.join(os.environ.get('prodir'),'waom1_tds.nc')
hr_tds = xr.open_dataset(hr_tds_path)

In [7]:
#lr_tds = hr_tds.drop(['tide_Eamp','tide_Ephase','tide_Cphase','tide_Cangle','tide_Cmin','tide_Cmax','lat_rho',
                       #'lon_rho','mask_rho'])
    
lr_tds ={}
gt = 'rho'
time = 'tide_period'
for var in ['tide_Ephase','tide_Cphase','tide_Cangle','tide_Cmin','tide_Cmax','tide_Eamp']:
    print('processing ',var)
    lr_tds[var] = resample_frc(hr_grd,lr_grd,hr_tds[var].to_dataset(),var,gt,time,write=False)
lr_tds = xr.merge([lr_tds['tide_Eamp'],lr_tds['tide_Ephase'],lr_tds['tide_Cphase'],lr_tds['tide_Cangle'],lr_tds['tide_Cmin'],lr_tds['tide_Cmax']])
out_path = os.path.join(os.environ.get('prodir'),'waom10_tds.nc')
lr_tds.to_netcdf(out_path)

processing  tide_Ephase


/home/ubuntu/bigStick/anaconda3/envs/tidal_melting/lib/python3.6/site-packages/pyresample/kd_tree.py:399: UserWarning: Possible more than 4 neighbours within 2000 m for some data points
  (neighbours, radius_of_influence))


VBox(children=(HTML(value=''), IntProgress(value=0, max=13)))

processing  tide_Cphase


VBox(children=(HTML(value=''), IntProgress(value=0, max=13)))

...

VBox(children=(HTML(value=''), IntProgress(value=0, max=13)))

processing  tide_Cmax


VBox(children=(HTML(value=''), IntProgress(value=0, max=13)))

processing  tide_Eamp


VBox(children=(HTML(value=''), IntProgress(value=0, max=13)))

In [ ]:
lr_tds